# Tensorflow Practice

In [3]:
import tensorflow as tf

# 상수노드 생성 (자주 사용 안됨)
node1 = tf.constant(10, dtype=tf.float32) 
node2 = tf.constant(20, dtype=tf.float32) 

# 수치연산
node3 = node1 + node2 

# 그래프 실행
sess = tf.Session() # 2.x버전에서는 session이 삭제됨
print(sess.run(node3)) # 30.0

30.0


In [2]:
# Data Flow Graph에 입력값 할당 시, placeholder를 이용
import tensorflow as tf

node1 = tf.placeholder(dtype=tf.float32) # 입력 파라미터를 "받아주는" 바구니
node2 = tf.placeholder(dtype=tf.float32)
node3 = node1 + node2

sess = tf.Session()
result = sess.run(node3, feed_dict={node1: 10, node2: 20})
print(result) # 30.0

30.0


# Simple Linear Regression
-----

## 1. Tensorflow

In [8]:
import numpy as np       # 자료구조
import pandas as pd      # 데이터핸들링
import tensorflow as tf  # 머신러닝 


# 1. Training Data Set - 입력 데이터
# 1차원 벡터 -> 2차원 Matrix
x_data = (np.array([1,2,3,4,5])).reshape(5,1)  # 독립변수 1개; 공부시간
t_data = (np.array([3,5,7,9,11])).reshape(5,1) # lable; 시험성적
print(x_data)


# 2. Data -> Data Flow Graph => placeholder 사용
# 입력 데이터가 2차원 이상일 경우, 차원을 명시해야 한다.
# placeholder: 예측 모델이 완성된 후에는 미지의 X를 입력하여 예측값을 도출하는 용도로 사용
# 따라서 shape=[None, 1]: 2차원 Matrix이고, 컬럼의 갯수가 한 개, 대신 레코드의 수는 상관 없음
X = tf.placeholder(shape=[None,1], dtype=tf.float32) 
T = tf.placeholder(shape=[None,1], dtype=tf.float32)

# 3. Hypothesis : 일차함수 (y = Wx + b)

# (3-1). Weight ,bias - 계속 변하는 값
# Simple Linear Regression이므로 독립변수가 한개니까 W도 한 개만 구하면 된다.
W = tf.Variable(tf.random.normal([1,1]), name='weight') # 랜덤한 초기값 
                                                        # 나중에 그래프가 반복적으로 수행되면서 값이 갱신됨
                                                        # [1,1]: 2차원 Matrix 형태로 W "한개" 구함

# [1]: 1차원 벡터 형태로 W "한개" 구함
# 행렬곱 연산을 안하므로 굳이 2차원 Matrix 형태로 구할 필요 없음
b = tf.Variable(tf.random.normal([1]), name='bias')  


# (3-2). Predict Model 정의 => 일차함수를 행렬식으로 변환: y = X dot W + b
## X, W: 2차원 Matrix
## b: 스칼라 (broadcasting)

H = tf.matmul(X, W) + b # y = Wx + b => 2차원 행렬로 처리 => y = X dot W + b
                        # y = 예측값, prediction model
    
# 4. W,b를 구하기 위해 평균제곱오차를 이용한 최소제곱법을 통해 손실함수 정의
loss = tf.reduce_mean(tf.square(H - T)) # 행렬차에 대해 각각의 값을 제곱 후 평균구함


# 5. 반복 학습 진행 - 손실함수를 미분해나가면서 W, b를 갱신 

# (5-1). 경사하강법 알고리즘을 수행할 학습용 노드 생성
#       : (미분을 통해) 손실함수의 값을 줄여가면서 W, b의 값을 1번 갱신
train = tf.train.GradientDescentOptimizer(learning_rate=1e-3).minimize(loss) 

# (5-2). 위의 (5-1)의 작업을 반복 수행하여 최적의 W, b를 구함 -> # 7으로 이동


# 6. Tensorflow 그래프 실행
sess = tf.Session() # Session 얻어옴
sess.run(tf.global_variables_initializer()) # 전역변수 초기화 (2.x 버전에서는 불필요)

 
# 7. 반복 학습 진행 - 1 epoch: Training Data Set 전체를 이용하여 한 번 학습
for step in range(30000): # 3000 epoch
    
    # 학습용 - train 노드만 실행하면 모든 하위노드들이 실행됨
    # 확인용 - 구해야하는 값: W, b 그리고 손실함수 값: 0과 가까워질 만큼 작아져야함
    _, W_val, b_val, loss_val  = sess.run([train, W, b, loss], feed_dict={X: x_data, T: t_data}) # _: 리턴값 사용 안함
    
    
    if step%3000 == 0: # 10번만 출력
        # y = 2x + 1, 따라서 W = 2, b = 1
        print('W: {}, b: {}, loss: {}'.format(W_val, b_val, loss_val))
        
        
# 8. 학습 종료 후 최적의 W, b가 계산되면 이를 이용하여 모델(H)이 완성됨
#    이제, Prediction(예측) 작업을 실행해보자.

result = sess.run(H, feed_dict={X: [[9]]}) # X: 2차원 Matrix 형태이므로 중첩 리스트(1행1열)로 구현
print('예측값: {}'.format(result)) # 18.999674

[[1]
 [2]
 [3]
 [4]
 [5]]
W: [[0.26164162]], b: [1.3429476], loss: 31.237262725830078
W: [[1.9230976]], b: [1.2776417], loss: 0.014040273614227772
W: [[1.9721142]], b: [1.1006749], loss: 0.0018460738938301802
W: [[1.9898862]], b: [1.0365115], loss: 0.00024281651712954044
W: [[1.9963293]], b: [1.0132507], loss: 3.198089325451292e-05
W: [[1.9986664]], b: [1.004817], loss: 4.2258411667717155e-06
W: [[1.9995133]], b: [1.001755], loss: 5.610386324406136e-07
W: [[1.9998204]], b: [1.0006487], loss: 7.667426871194039e-08
W: [[1.9999334]], b: [1.0002346], loss: 1.008385197565076e-08
W: [[1.9999406]], b: [1.0002079], loss: 7.925575751244196e-09
예측값: [[18.999674]]


## 2. Python

In [12]:
import numpy as np

# 1. Training Data Set
x_data = np.array([1,2,3,4,5]).reshape(5,1)
t_data = np.array([3,5,7,9,11]).reshape(5,1)

# 2. Weight, bias
W = np.random.rand(1,1) # 2차 Matrix: [[0.32456]]
b = np.random.rand(1)   # 스칼라

# 3. Hypothesis - 모델 학습이 종료된 후 모델이 생성되고 나서, 예측값을 구할 때 사용
def predict(x):
    
    y = np.dot(x, W) + b # y = X dot W + b
    return y

# 4. Loss Function: W와 b의 함수이므로 W와 b를 인자로 받아서 손실함수를 계산
#                   그런데, 수치미분을 위해 W, b를 하나의 리스트 안에 넣어서 하나의 인자로 받음

# 아래의 함수는 W, b가 한 개인 경우로 Hard Code했기 때문에 Simple Linear Regression에서만 사용 가능
def loss_func(input_obj):
    
    # input_obj = [W, b]
    input_W = input_obj[0]
    input_b = input_obj[1]
    
    y = np.dot(x_data, input_W) + input_b     # Hypothesis
    
    return np.mean(np.power((t_data - y), 2)) # 손실함수

# 5. 특정 W, b에서 손실함수를 편미분 -> 최적의 W, b?
# 다변수 함수(W, b)에 대한 수치미분 코드 사용
def numerical_derivative(f, x):
    
    # f: 편미분하려고 하는 다변수 함수 -> 손실함수
    # x: 편미분하려고 하는 모든 값 -> W, b
    # 따라서, [W, b]에 대해 각각 편미분이 진행
    
    delta_x = 1e-4
    derivative_x = np.zeros_like(x) 
    
    it = np.nditer(x, flags=['multi_index'])
    
    while not it.finished: 
        idx = it.multi_index 
        # print('현재의 idx: {}'.format(idx)) 

        tmp = x[idx]                        
        # print('현재의 tmp: {}'.format(tmp)) # 1.0 # 2.0
        
        # x에 대한 편미분
        x[idx] = tmp + delta_x 
        fx_plus_delta = f(x) 
        
        # 중앙차분 미분 준비
        x[idx] = tmp - delta_x
        fx_minus_delta = f(x) 
        
        # 중앙차분
        derivative_x[idx] = (fx_plus_delta - fx_minus_delta) / (2 * delta_x)
        
        # 두번째 독립변수에 대해 편미분 시행을 위해 원래 상태로 복구
        x[idx] = tmp 
    
        it.iternext() # 다음 iterator, 다음 칸으로 넘어감
        
    return derivative_x



# 6. learning rate 설정
learning_rate = 1e-4


# 7. 반복 학습 진행: 손실함수 미분 -> W, b 갱신
for step in range(300000):
    
    # 현재 W, b값을 입력 파라미터 값으로 저장하여 편미분 함수에 인자값으로 넘기기 위한 용도
    # [W b]
    input_param = np.concatenate((W.ravel(), b.ravel()), axis=0) # 1차원 벡터끼리 연결하기 위해 ravel 사용
    
    # 손실함수를 input_param에 대해 편미분 시행
    derivative_result = learning_rate * numerical_derivative(loss_func, input_param)
    
    # W, b 갱신
    # 위에서 합쳐진 인자를 각각 뽑아서 계산
    W = W - derivative_result[:1].reshape(1, 1) # 2차원 - 2차원
    b = b - derivative_result[1:] # 1차원 벡터
    
    if step%30000 == 0:
        print('W: {}, b: {}'.format(W, b)) 
    

W: [[0.58561806]], b: [0.40027785]
W: [[2.0194011]], b: [0.92995587]
W: [[2.00703589]], b: [0.97459821]
W: [[2.00255159]], b: [0.99078794]
W: [[2.00092535]], b: [0.99665921]
W: [[2.00033558]], b: [0.99878845]
W: [[2.0001217]], b: [0.99956063]
W: [[2.00004413]], b: [0.99984066]
W: [[2.00001601]], b: [0.99994221]
W: [[2.0000058]], b: [0.99997904]


## 3. Sklearn

In [14]:
import numpy as np
from sklearn import linear_model

# 1. Training Data Set
x_data = np.array([1,2,3,4,5]).reshape(5,1)
t_data = np.array([3,5,7,9,11]).reshape(5,1)

# 2. Linear Regression Model 생성
model = linear_model.LinearRegression()

# 3. 학습 진행 -> 모델 생성
model.fit(x_data, t_data)

# 4. Weight, bias 출력
print('W: {}, b: {}'.format(model.coef_, model.intercept_)) # W: [[2.]], b: [1.]

# 5. Prediction
print(model.predict([[9]])) # [[19.]]

W: [[2.]], b: [1.]
[[19.]]
